In [1]:
import pandas as pd

### Extract methodology fields for each year

In [2]:
mf = pd.read_excel('/Users/cave/Desktop/discriminology/methodology_fields.xlsx')

In [377]:
fields_1112 = mf['Field_Name 2011/12'].str.replace('\n', ',').str.split(',').apply(pd.Series).stack().reset_index(drop=True).values
fields_1112 = [s.strip() for s in list(set(fields_1112))]

fields_1314 = mf['Field_Name 2013/14'].str.replace('\n', ',').str.split(',').apply(pd.Series).stack().reset_index(drop=True).values
fields_1314 = list(set(fields_1314))

fields_1516 = mf['Field_Name 2015/16'].str.replace('\n', ',').str.split(',').apply(pd.Series).stack().reset_index(drop=True).values
fields_1516 = list(set(fields_1516))

### Create helper functions to aggregate dataframes and label coded columns

In [384]:
def aggregate_data(frame_array, desired_fields):

    clean_frames = []

    for df in frame_array:
        # subset dataframe to desired columns
        temp = df[list(set(df.columns[df.columns.isin(desired_fields)]))]
        temp.columns = [c.strip() for c in temp.columns]

        num = temp._get_numeric_data()
        num[num < 0] = 0 # replace negative numbers with 0
        temp = temp.replace('<=2', '2') # remove misc symbols from values
        temp = temp.replace('‡', None) # remove misc symbols from values
        temp['COMBOKEY'] = temp['COMBOKEY'].astype(str)
        temp.set_index('COMBOKEY', inplace=True)
        clean_frames.append(temp)


    concat = pd.concat(clean_frames, axis=1, sort=True)
    flipped = concat.T.drop_duplicates()
    grouped = flipped.groupby(lambda x: x).agg({c: 'last' for c in flipped.columns})
    return grouped.T

In [390]:
df1_desc_1112.head()

,Field Name,Field Description
0,LEA_STATE,District State
1,LEAID,7 Digit LEAID District Identification Code
2,LEA_NAME,District Name
3,SCHID,5 Digit School Identification Code
4,SCH_NAME,School Name


In [393]:
def join_col_descriptions(frames, agg_data, fields, year):
    '''
    INPUTS
    
    frames (list of DataFrames): Array of dataframes with field codes and descriptions
    agg_data (DataFrame): Aggregated data with coded fields as columns
    year (STR): year range of data e.g. '2015-16'
    
    '''
    descriptions = pd.concat(frames, axis=0)
    descriptions['Field Name'] = descriptions['Field Name'].str.strip()
    descriptions.drop_duplicates(inplace=True)
    descriptions.set_index('Field Name', inplace=True)
    descriptions = descriptions.loc[fields]
    descriptions.columns = ['description']
    final = pd.merge(agg_data.T, descriptions, left_index=True, right_index=True, how='left').set_index('description', append=True).T
    final['YEAR','School Year'] = year
    final.to_csv(f'~/Desktop/discriminology/output/final_data_{year}.csv')
    return final
    

### Load all data + col description files for 2011-12

In [4]:
## Reads originals from excel

# df1_1112, df1_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/05 - Overall Enrollment.xlsx', sheet_name=None).values()
# df2_1112, df2_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/08 - Students enrolled in Gifted-Talented Programs.xlsx', sheet_name=None).values()
# df3_1112, df3_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/10-1 - Students with Disabilities Served under IDEA Enrollment.xlsx', sheet_name=None).values()
# df4_1112, df4_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/10-2 - Students with Disabilities Served under 504 Enrollment.xlsx', sheet_name=None).values()
# df5_1112, df5_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/W:O Disabilities/35-3 - Students WO Disab Receiving only one out-of-school suspension.xlsx', sheet_name=None).values()
# df6_1112, df6_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/W:O Disabilities/35-4 - Students WO Disab Rec more than one out-of-school suspension.xlsx', sheet_name=None).values()
# df7_1112, df7_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/With Disabilities/36-3 - Students With Disabilities Receiving only one out-of-school suspension.xlsx', sheet_name=None).values()
# df8_1112, df8_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Out of School Suspensions/With Disabilities/36-4 - Students With Disab Receiving more than one out-of-school suspension.xlsx', sheet_name=None).values()
# df9_1112, df9_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Academic/Advanced Placement/17 - Students who are taking at least one AP course.xlsx', sheet_name=None).values()
# df10_1112, df10_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Referral to law enforcement/W:O Disabilities/35-8 - Students Without Disabilities Referral to law enforcement.xlsx', sheet_name=None).values()
# df11_1112, df11_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/Referral to law enforcement/With Disabilities/36-8 - Students With Disabilities Referral to law enforcement.xlsx', sheet_name=None).values()
# df12_1112, df12_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/School Related Arrest/W:O Disabilities/35-9 - Students Without Disabilities School-related arrest.xlsx', sheet_name=None).values()
# df13_1112, df13_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Discipline/School Related Arrest/With Disabilities/36-9 - Students With Disabilities School-related arrest.xlsx', sheet_name=None).values()
# df14_1112, df14_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/School Characteristics/02 - School Characteristics.xlsx', sheet_name=None).values()
# df15_1112, df15_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Staff/08-1 School Support and Security Staff (required elements).xlsx', sheet_name=None).values()
# df16_1112, df16_desc_1112 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2011:12/Enrollment/06 - Enrolled in Early Childhood and Prekindergarten.xlsx', sheet_name=None).values()

In [13]:
# ctr = 0
# for frame in desc_frames_1112:
#     frame.to_csv(f'/Users/cave/Desktop/discriminology/2011_12/descriptions/file_{ctr}.csv')
#     ctr += 1


In [123]:
df1_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_1.csv')
df2_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_2.csv')
df3_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_3.csv')
df4_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_4.csv')
df5_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_5.csv')
df6_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_6.csv')
df7_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_7.csv')
df8_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_8.csv')
df9_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_9.csv')
df10_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_10.csv')
df11_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_11.csv')
df12_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_12.csv')
df13_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_13.csv')
df14_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_14.csv')
df15_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_15.csv')
df16_1112 = pd.read_csv('~/Desktop/discriminology/2011_12/data/file_0.csv')

/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8,9,10,11,12,13,14,17,18,19,20,21,22,23,24,25,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [124]:

frames_1112 = [df1_1112, df2_1112, df3_1112, df4_1112, df5_1112, df6_1112,
               df7_1112, df8_1112, df9_1112, df10_1112, df11_1112, df12_1112,
               df13_1112, df14_1112, df15_1112, df16_1112]



desc_frames_1112 = [df1_desc_1112, df2_desc_1112, df3_desc_1112, df4_desc_1112,
                    df5_desc_1112, df6_desc_1112, df7_desc_1112, df8_desc_1112,
                    df9_desc_1112, df10_desc_1112, df11_desc_1112, df12_desc_1112,
                    df13_desc_1112, df14_desc_1112, df15_desc_1112, df16_desc_1112]


Use helper function to aggregate frames and clean up negative values, standardize length of zipcodes.

In [385]:
df_1112 = aggregate_data(frames_1112, fields_1112)
df_1112['SCH_FTESECURITY_IND'] = df_1112['SCH_FTESECURITY_IND'].str.replace('-9','No')
df_1112.columns = [c.strip() for c in df_1112.columns]
df_1112['SCH_ZIP'] = df_1112['SCH_ZIP'].apply(lambda x: str(x).zfill(5))

df_1112_final = join_col_descriptions(desc_frames_1112, df_1112, fields_1112, '2011-12')

/Users/cave/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  del sys.path[0]


In [394]:
df_1112_final = join_col_descriptions(desc_frames_1112, df_1112, fields_1112, '2011-12')

Custom case handling: Juvenile Justice facilities

In [314]:
df_1112_final['JJ', 'Juvenile Justice Facility - Z: Long term secure facility, X: Other JJ facility'] = df_1112_final['JJ', 'Juvenile Justice Facility - Z: Long term secure facility, X: Other JJ facility'].str.replace('Z', 'Yes').str.replace('X', 'Yes')
df_1112_final['JJ', 'Juvenile Justice Facility: "Yes" indicates a long-term secure facility; "No" indicates not a JJ facility'] = df_1112_final['JJ', 'Juvenile Justice Facility - Z: Long term secure facility, X: Other JJ facility']
df_1112_final.drop(columns=['Juvenile Justice Facility - Z: Long term secure facility, X: Other JJ facility']
                   ,level=1
                   ,inplace=True
                  )

In [315]:
df_1112_final[::100].to_csv('~/Desktop/discriminology/output/11_12_sample.csv')

In [214]:
missing_fields = pd.Series(fields_1112)[~pd.Series(fields_1112).isin(df_1112.columns)].values
assert missing_fields ==  'COMBOKEY'

### Load all data + col description files for 2013-14

Collect column name descriptions from each spreadsheet

In [188]:
# # Read originals from excel

# df1_1314, df1_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/01 School Characteristics.xlsx', sheet_name=None).values()
# df2_1314, df2_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/03 Enrollment.xlsx', sheet_name=None).values()
# df3_1314, df3_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/04-1 Gifted and Talented Enrollment.xlsx', sheet_name=None).values()
# df4_1314, df4_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/06 Advanced Placement and International Baccalaureate Diploma Programme Enrollment.xlsx', sheet_name=None).values()
# df5_1314, df5_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/07-2 Advanced Placement Exams.xlsx', sheet_name=None).values()
# df6_1314, df6_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/08-1 School Support and Security Staff (required elements).xlsx', sheet_name=None).values()
# df7_1314, df7_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/09-1 Chronic Absenteeism.xlsx', sheet_name=None).values()
# df8_1314, df8_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/11-2 Suspensions (required elements).xlsx', sheet_name=None).values()
# df9_1314, df9_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/11-3 Expulsions.xlsx', sheet_name=None).values()
# df10_1314, df10_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/12 Student Referrals and Arrests.xlsx', sheet_name=None).values()
# df11_1314, df11_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/16 School Expenditures.xlsx', sheet_name=None).values()
# df12_1314, df12_desc_1314 = pd.read_excel('~/Desktop/discriminology/OCR School data sample/2013:14/CRDC-collected data file for Schools/17 Justice Facilities.xlsx', sheet_name=None).values()

In [189]:
frames_1314 = [df1_1314, df2_1314, df3_1314, df4_1314, df5_1314,
               df6_1314, df7_1314, df8_1314, df9_1314, df10_1314,
               df11_1314, df12_1314]


desc_frames_1314 = [df1_desc_1314, df2_desc_1314, df3_desc_1314,
                    df4_desc_1314, df5_desc_1314, df6_desc_1314,
                    df7_desc_1314, df8_desc_1314, df9_desc_1314,
                    df10_desc_1314, df11_desc_1314, df12_desc_1314]

In [191]:
# ctr = 0
# for frame in desc_frames_1314:
#     frame.to_csv(f'~/Desktop/discriminology/2013_14/descriptions/file_{ctr}.csv')
#     ctr += 1


# ctr = 0
# for frame in frames_1314:
#     frame.to_csv(f'~/Desktop/discriminology/2013_14/data/file_{ctr}.csv')
#     ctr += 1


In [193]:
df1_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_1.csv')
df2_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_2.csv')
df3_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_3.csv')
df4_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_4.csv')
df5_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_5.csv')
df6_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_6.csv')
df7_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_7.csv')
df8_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_8.csv')
df9_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_9.csv')
df10_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_10.csv')
df11_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_11.csv')
df12_1314 = pd.read_csv('~/Desktop/discriminology/2013_14/data/file_0.csv')

In [317]:
df_1314 = aggregate_data(frames_1314, fields_1314)

In [328]:
df_1314['SCH_FTESECURITY_IND'] = df_1314['SCH_FTESECURITY_IND'].str.replace('-9','No').str.replace('-5','No')
df_1314 = df_1314.replace('-5', None)
df_1314 = df_1314.replace('-9', None)

In [330]:
df_1314_final = join_col_descriptions(desc_frames_1314, df_1314, fields_1314, '2013-14')

In [331]:
df_1314_final[::100].to_csv('~/Desktop/discriminology/output/13_14_sample.csv')

In [325]:
missing_fields = pd.Series(fields_1314)[~pd.Series(fields_1314).isin(df_1314.columns)].values
assert missing_fields ==  'COMBOKEY'

### Get column decriptions for the 2015-16 data

In [229]:
col_descr_1516 = pd.read_excel('/Users/cave/Desktop/discriminology/OCR School data sample/2015:16/CRDC 2015-16 School Data Record Layout copy.xlsx', index_col='Field_Name')
decoded_names = pd.DataFrame(col_descr_1516.loc[fields_1516]['Field_Description'])
decoded_names.columns = ['description']

In [230]:
decoded_names.reset_index().to_csv('/Users/cave/Desktop/2015_16_field_descriptions.csv')

### Isolate numeric columns in 15/16 and replace negative numbers with zeroes.

In [219]:
df1516_raw = pd.read_csv('/Users/cave/Desktop/discriminology/OCR School data sample/2015:16/CRDC 2015-16 School Data copy.csv'
                      , encoding='iso-8859-1'
                     )
df1516_raw['COMBOKEY'] = df1516_raw['LEAID'].astype(str) + df1516_raw['SCHID'].astype(str).str.zfill(5)
df1516_raw = df1516_raw[list(set(fields_1516))]

/Users/cave/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,6,23,24,25,30,31,1828) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Replace negative values with zeroes.

In [222]:
num = df1516_raw._get_numeric_data()
num[num < 0] = 0
df1516_raw = df1516_raw.set_index(['COMBOKEY'])

In [231]:
df1516_final = pd.merge(df1516_raw.T, decoded_names, left_index=True, right_index=True).set_index('description', append=True).T
df1516_final['YEAR','School Year'] = '2015-16'

In [241]:
missing_fields = pd.Series(fields_1516)[~pd.Series(fields_1516).isin(df1516_raw.columns)].values
assert missing_fields ==  'COMBOKEY'
assert len(df1516_final) == df1516_final.index.nunique()

In [333]:
df1516_final[::100].to_csv('~/Desktop/discriminology/output/15_16_sample.csv')

In [334]:
df1516_final.to_csv('~/Desktop/discriminology/output/final_data_2015-16.csv')

### Code Sandbox - everything below only needs to be run once

In [ ]:
# descriptions = pd.concat(desc_frames_1112, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1112]
# descriptions.columns = ['2011_12_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2011_12_field_descriptions.csv')

# descriptions = pd.concat(desc_frames_1314, axis=0)
# descriptions.drop_duplicates(inplace=True)
# descriptions.set_index('Field Name', inplace=True)
# descriptions = descriptions.loc[fields_1314]
# descriptions.columns = ['2013_14_description']
# descriptions.reset_index().to_csv('/Users/cave/Desktop/2013_14_field_descriptions.csv')